In [1]:
import numpy as onp
import pylab as plt
import tqdm

import imageio
import os
from skimage.transform import rescale, resize, downscale_local_mean
import pandas as pd
import time
import scipy
from numpy.linalg import norm


In [2]:
cities= onp.load("cities.npy", allow_pickle=True)
cities

array([[0.42063307, 0.36365371],
       [0.42055583, 0.71203725],
       [0.77196185, 0.52303056],
       ...,
       [0.81118992, 0.84354499],
       [0.78685968, 0.50938438],
       [0.83058875, 0.35190042]])

In [3]:
# creating distance matrix 
dist=norm(cities-cities[0],axis=1)
for i in range(1,len(cities)):
    temp=norm(cities-cities[i],axis=1)
    dist=onp.c_[dist,temp]

print(dist.shape) # 1000 by 1000

(1000, 1000)


# Nearest insertion algo
sources : 
https://datascience.lc/travelling-salesman-problem-tsp-concept/ 
https://github.com/afourmy/pyTSP
https://stemlounge.com/animated-algorithms-for-the-traveling-salesman-problem/

In [4]:
def insert_nearest(tour,k,cities):
    '''
    To insert city k into tour
    Basically minimize function of dist(i,k)+dist(k,j)-dist(i,j), where i and j are the cities currently in tour
    cities is the coordinates of all cities to travel
    '''

    min_dist=999999999
    insert_pos=None
    for i in range(len(tour)):
        dist_ij=norm(cities[tour[i%len(tour)]]-cities[tour[(i+1)%len(tour)]])
        dist_ik=norm(cities[tour[i%len(tour)]]-cities[k])
        dist_kj=norm(cities[k]-cities[tour[(i+1)%len(tour)]])
        dist=dist_ik+dist_kj-dist_ij
        #print(i)
        #print(dist)
        if dist <min_dist : # only add in if the 
            insert_pos=i+1
            min_dist=dist
        #print(min_dist)
    #print(insert_pos)
    # when tour has only 2 cities, and when inserting in between the 2 cities lead to a longer route
    #choose to place them at either the start or end
    if len(tour)==2 and min_dist>0:
        start_dis=norm(cities[k]-cities[tour[0]])
        end_dis=norm(cities[k]-cities[tour[1]])
        #print(start_dis)
        #print(end_dis)
        if end_dis<start_dis:
            tour.append(k)
        else:
            tour.insert(0,k)
        return
    tour.insert(insert_pos,k)

        
        
        


In [5]:
def select_k(tour,left,dist):
    sel_dist=dist[:,left][tour,]
    k=left[sel_dist.argmin()%len(left)]
    left.remove(k)
    return k


In [6]:
def tsp_nearest_insertion(city1,cities,dist):
    #intialise left list 
    left=list(range(len(cities)))

    left.remove(city1)
    #2nd city is the one nearest to city 1
    city2=left[dist[city1,left].argmin()]
    left.remove(city2)
    tour=[city1,city2]

    while len(tour)<len(cities):
        k=select_k(tour,left,dist) #select next city based on the nearest city to any city in current tour list
        insert_nearest(tour,k,cities)

    nearest_insert_total_dist=sum(norm(cities[tour[i]]-cities[tour[i+1]]) for i in range(len(tour)-1))
    return nearest_insert_total_dist,tour

In [9]:
# time needed for one run

start=time.time()
tsp_nearest_insertion(1,cities,dist)
end=time.time()
end-start # about 15 secs 

15.666501998901367

In [15]:
# use every possible city as city 1 and store respective L 
# takes about 4-5h to run 
L_lst_nearest=[]
tour_lst_nearest=[]
for i in tqdm.tqdm(range(len(cities))):
    L_cityi,tour_cityi=tsp_nearest_insertion(i,cities,dist)
    L_lst_nearest.append(L_cityi)
    tour_lst_nearest.append(tour_cityi)



100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [2:37:14<00:00,  9.43s/it]


In [56]:
#find min L and respective city
L_arr_nearest=onp.array(L_lst_nearest)
L_min_nearest=L_arr_nearest.min() # smallest L
city_min_nearest=L_arr_nearest.argmin() # respective city 
tour_min_nearest=tour_lst_nearest[city_min_nearest] #respective permutation 

print(L_min_nearest)
print(city_min_nearest)
print(tour_min_nearest)

28.701855065308923
588
[463, 265, 305, 588, 993, 342, 248, 122, 922, 475, 451, 664, 218, 688, 571, 864, 985, 174, 204, 125, 965, 990, 402, 395, 709, 983, 198, 257, 668, 502, 569, 911, 853, 915, 110, 663, 454, 862, 920, 77, 484, 193, 893, 349, 821, 274, 518, 962, 1, 544, 809, 459, 180, 827, 468, 787, 754, 370, 439, 734, 908, 559, 600, 969, 165, 230, 93, 956, 6, 350, 760, 195, 907, 693, 286, 513, 30, 510, 241, 782, 921, 939, 380, 914, 634, 980, 812, 471, 978, 205, 902, 344, 393, 717, 424, 696, 676, 379, 118, 413, 240, 873, 243, 400, 325, 358, 626, 741, 292, 34, 472, 715, 94, 538, 121, 478, 795, 434, 221, 887, 46, 799, 997, 442, 611, 136, 567, 607, 48, 731, 623, 314, 881, 595, 433, 699, 114, 508, 490, 52, 483, 438, 743, 604, 777, 181, 9, 704, 556, 796, 503, 457, 776, 301, 669, 115, 25, 803, 239, 41, 494, 723, 599, 324, 516, 830, 59, 275, 774, 445, 15, 157, 73, 83, 95, 427, 714, 98, 100, 49, 108, 721, 199, 612, 259, 823, 7, 919, 321, 552, 900, 807, 888, 97, 135, 560, 329, 587, 897, 973, 75

# Cheapet insert algo

In [10]:
def min_dist_k(tour,k,cities):
    '''
    Finds out the min distance when city k is inserted and the relative position
    '''
    min_dist=999999999
    insert_pos=None
    for i in range(len(tour)):
        dist_ij=norm(cities[tour[i%len(tour)]]-cities[tour[(i+1)%len(tour)]])
        dist_ik=norm(cities[tour[i%len(tour)]]-cities[k])
        dist_kj=norm(cities[k]-cities[tour[(i+1)%len(tour)]])
        dist=dist_ik+dist_kj-dist_ij
        #print(i)
        #print(dist)
        if dist <min_dist : # only add in if the 
            insert_pos=i+1
            min_dist=dist
        #print(min_dist)
    #print(insert_pos)
    #when tour has only 2 cities, the min distance calculated is actually the same regardless of the position to insert k at
    #hence insert it at the end if the min distance is poistive (adding city k lengthens the path)
    if len(tour)==2 and min_dist>0:
        insert_pos=len(tour)

    return [min_dist,insert_pos]

        

In [11]:
def insert_cheapest(tour,left,cities):
    dist_pos=[] #build a matrix to store the min distance, postion each city is inserted into, and number of the city
    #used to compare which city is the next to be inserted 
    
    for i in left:
        temp=min_dist_k(tour,i,cities)
        temp.append(i)
        dist_pos.append(temp)
    dist_pos_array=onp.array(dist_pos) #change into array
    #print(dist_pos_array)
    x=dist_pos_array[dist_pos_array.argmin(axis=0)[0],:]
    #print(x)
    tour.insert(int(x[1]),int(x[2]))
    left.remove(int(x[2]))
    
    
    

In [12]:
def tsp_cheapest_insertion(city1,cities,dist):
    #intialise left list 
    left=list(range(len(cities)))

    left.remove(city1)
    #2nd city is the one nearest to city 1
    city2=left[dist[city1,left].argmin()]
    left.remove(city2)
    tour=[city1,city2]
    #iteration=0
    while len(tour)<len(cities):
        #iteration+=1
        insert_cheapest(tour,left,cities)
    cheapest_insert_total_dist=sum(norm(cities[tour[i]]-cities[tour[i+1]]) for i in range(len(tour)-1))
    return cheapest_insert_total_dist,tour

In [13]:
# using the starting city that produces the lowest L in nearest insertion as starting city of cheapest insertion 
#city1=588
start=time.time()
L_588_cheapest,tour_588_cheapest=tsp_cheapest_insertion(588,cities,dist)
end=time.time()
(end-start)/60 #about 74mins 

74.35237829287847

In [ ]:
#time in seconds 
(end-start)

In [55]:
print(L_588_cheapest)
print(tour_588_cheapest)
#performs better than nearest insertion 
#but took a longer time 

27.74670354295936
[588, 993, 342, 248, 122, 922, 475, 451, 664, 218, 688, 864, 985, 174, 571, 990, 204, 125, 965, 402, 907, 195, 395, 709, 983, 198, 257, 668, 502, 569, 911, 853, 915, 110, 663, 454, 920, 283, 77, 484, 193, 893, 349, 821, 274, 518, 962, 1, 83, 95, 919, 73, 559, 286, 693, 600, 969, 760, 350, 6, 956, 93, 230, 165, 544, 809, 459, 180, 468, 827, 787, 754, 118, 413, 240, 445, 15, 874, 635, 109, 275, 59, 830, 774, 873, 379, 676, 400, 25, 803, 239, 41, 743, 438, 483, 52, 490, 508, 114, 160, 378, 212, 732, 589, 322, 339, 563, 163, 4, 57, 987, 937, 336, 865, 885, 988, 647, 391, 691, 208, 886, 356, 646, 689, 436, 75, 323, 482, 392, 514, 840, 498, 319, 264, 525, 547, 224, 79, 506, 766, 55, 217, 27, 313, 869, 396, 967, 615, 521, 783, 348, 213, 598, 330, 671, 903, 404, 23, 353, 88, 149, 351, 537, 561, 256, 414, 293, 255, 710, 699, 433, 595, 881, 314, 623, 731, 494, 48, 607, 567, 136, 611, 442, 997, 799, 46, 887, 434, 795, 221, 905, 752, 387, 117, 317, 711, 53, 756, 831, 828, 578, 13